In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import re
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
os.getcwd()


'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play1'

In [4]:
df = pd.read_csv('./daily_ou_kpi.csv') 
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
# df.to_csv('./daily_kpi_all_810.csv', encoding='utf_8_sig')

clean_df1 = (df.groupby('ou_code')['operation_day'].count() <2).reset_index()
clean_df1.columns = ['ou_code', 'flag1']
df = clean_df1.merge(df, on = 'ou_code', how = 'inner')
df = df[df['flag1'] == False]

clean_df2 = df.groupby('ou_code')[[
    'inbound_receive_qty', 'outbound_shipped_qty'
    ]].sum().reset_index()
clean_df2['sum'] = clean_df2.sum(axis = 1)
clean_df2 = clean_df2[clean_df2['sum'] != 0]
df = df[df['ou_code'].isin(clean_df2.ou_code)]

clean_df3 = (df.groupby('ou_code')[[
    'total_working_hour'
    ]].sum() == 0).reset_index()
clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]
# clean_df4 = (df.groupby('ou_code')[[
#     'outsource_working_hour'
#     ]].sum() == 0).reset_index()
# clean_df4 = clean_df4[clean_df4['outsource_working_hour'] == False]
# df = df[df['ou_code'].isin(clean_df4.ou_code)]
# df= df.reset_index()
 
# df = df[df['ou_code'].isin(clean_df3.ou_code)]
# df= df.reset_index()

df = df[[
    'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
    'outbound_shipped_qty','total_head_count','total_working_hour',
    'outsource_working_hour', 'perm_working_hour',
    'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
    'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
    'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]
df = df.fillna(0)
df = df[df['total_working_hour'] != 0]
df.head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
0,CN-001,20210430,0.0,0,1699.0,4.0,32.75,0.0,0.00,0.0,32.75,0.0,0.000000,0.0,8.1875,0.000000,0.000000
1,CN-001,20210619,0.0,1,0.0,3.0,30.00,0.0,30.00,0.0,30.00,0.0,0.000000,1.0,10.0000,0.011628,0.988372
2,CN-001,20210427,4794.0,0,7392.0,4.0,32.28,0.0,0.00,0.0,32.28,0.0,1.541927,0.0,8.0700,0.000000,0.000000
3,CN-001,20210419,11400.0,0,6000.0,4.0,33.25,0.0,0.00,0.0,33.25,0.0,0.526316,0.0,8.3125,0.000000,0.000000
4,CN-001,20210617,114.0,0,13527.0,5.0,40.58,0.0,40.58,0.0,40.58,0.0,118.657895,1.0,8.1160,0.001600,0.998400


In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
res = smf.ols(
    'total_working_hour ~ inbound_receive_qty + outbound_shipped_qty', 
    data=df).fit()
res.summary()

df['operation_day'] = pd.to_datetime(df['operation_day'].apply(str), yearfirst = True)
df.set_index(['ou_code', 'operation_day'], inplace=True)
df['ou_codes'] = df.index.get_level_values(0)  

from linearmodels import PanelOLS
# Regression
FE = PanelOLS(df['total_working_hour'], df[['inbound_receive_qty','outbound_shipped_qty']],
            entity_effects = True,
            time_effects = True,
            check_rank =True,
              )
            
# Result
result = FE.fit(
  cov_type = 'clustered',\
  cluster_entity=True,
  cluster_time=True
            )
result

# dkrei predicitoonS

In [15]:
from mlxtend.regressor import StackingRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
data_all = df.copy();data_all.head(7)

inbound_receive_qty  is_holiday  outbound_shipped_qty  \
ou_code operation_day                                                          
CN-001  2021-04-30                     0.0           0                1699.0   
        2021-06-19                     0.0           1                   0.0   
        2021-04-27                  4794.0           0                7392.0   
        2021-04-19                 11400.0           0                6000.0   
        2021-06-17                   114.0           0               13527.0   
        2021-04-15                 17922.0           0                1591.0   
        2021-07-21                     0.0           0                1542.0   

                       total_head_count  total_working_hour  \
ou_code operation_day                                         
CN-001  2021-04-30                  4.0               32.75   
        2021-06-19                  3.0               30.00   
        2021-04-27                  4.0               32.28   
        2021-04-19                  4.0               33.25   
        2021-06-17                  5.0               40.58   
        2021-04-15                  4.0               34.75   
        2021-07-21                  5.0               45.17   

                       outsource_working_hour  perm_working_hour  \
ou_code operation_day                                              
CN-001  2021-04-30                        0.0               0.00   
        2021-06-19                        0.0              30.00   
        2021-04-27                        0.0               0.00   
        2021-04-19                        0.0               0.00   
        2021-06-17                        0.0              40.58   
        2021-04-15                        0.0               0.00   
        2021-07-21                        0.0              45.17   

                       other_working_hour  direct_working_hour  \
ou_code operation_day                                            
CN-001  2021-04-30                    0.0                32.75   
        2021-06-19                    0.0                30.00   
        2021-04-27                    0.0                32.28   
        2021-04-19                    0.0                33.25   
        2021-06-17                    0.0                40.58   
        2021-04-15                    0.0                34.75   
        2021-07-21                    0.0                45.17   

                       indirect_working_hour  outbound_inbound_qty_ratio  \
ou_code operation_day                                                      
CN-001  2021-04-30                       0.0                    0.000000   
        2021-06-19                       0.0                    0.000000   
        2021-04-27                       0.0                    1.541927   
        2021-04-19                       0.0                    0.526316   
        2021-06-17                       0.0                  118.657895   
        2021-04-15                       0.0                    0.088774   
        2021-07-21                       0.0                    0.000000   

                       perm_working_hour_ratio  working_hour_per_head  \
ou_code operation_day                                                   
CN-001  2021-04-30                         0.0                 8.1875   
        2021-06-19                         1.0                10.0000   
        2021-04-27                         0.0                 8.0700   
        2021-04-19                         0.0                 8.3125   
        2021-06-17                         1.0                 8.1160   
        2021-04-15                         0.0                 8.6875   
        2021-07-21                         1.0                 9.0340   

                       location_usage_rate  location_idle_rate ou_codes  
ou_code operation_day                                                    
CN-001  2021-04-30                0

In [17]:
from sklearn.model_selection import train_test_split  
train, test  = train_test_split(data_all, \
    test_size=0.20, random_state=42, stratify = data_all['ou_codes'])
train, valid = train_test_split(train, \
    test_size=0.20, random_state=42, stratify = train['ou_codes'])

In [18]:
def go_dummie(df):
    df = pd.concat([df.drop('ou_codes', axis =1) , pd.get_dummies(df['ou_codes'])], axis =1)
    return df
train, valid, test = [go_dummie(i) for i in [train, valid, test]]


In [19]:
train_y = train.pop('total_working_hour')
valid_y = valid.pop('total_working_hour')

In [20]:
import time #implementing in this function the time spent on training the model
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

def modelfit(alg, dtrain, target, only_predict = False):
    #Fit the algorithm on the data
    time_start = time.perf_counter() #start counting the time
    if not only_predict:
        alg.fit(dtrain, target)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    
    kfolds = KFold(n_splits=6, shuffle=True, random_state=42)
    
    cv_score = cross_val_score(alg, dtrain,target, cv=kfolds, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(-cv_score)
    
    time_end = time.perf_counter()
    
    total_time = time_end-time_start
    #Print model report:
    print("\nModel Report")
    print("RMSE :  {:.4f}".format(np.sqrt(mean_squared_error(target, dtrain_predictions))))
    print("CV Score : Mean -  %.4f | Std -  %.4f | Min -  %.4f | Max - %.4f" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    print("Amount of time spent during training the model and cross validation: %4.3f seconds" % (total_time))
    
def plot_feature_importance(model, df):
    feature_importance = model.feature_importances_[:30]
    # make importances relative to max importance
    plt.figure(figsize=(20, 20)) #figure size
    feature_importance = 100.0 * (feature_importance / feature_importance.max()) #making it a percentage relative to the max value
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, df.columns[sorted_idx], fontsize=15) #used train_drop here to show the name of each feature instead of our train_prepared 
    plt.xlabel('Relative Importance', fontsize=20)
    plt.ylabel('Features', fontsize=20)
    plt.title('Variable Importance', fontsize=30)

In [23]:
lin_reg = Ridge()
modelfit(lin_reg, train, train_y)


Model Report
RMSE :  0.0000
CV Score : Mean -  0.0000 | Std -  0.0000 | Min -  0.0000 | Max - 0.0000
Amount of time spent during training the model and cross validation: 0.313 seconds


In [24]:
from sklearn.linear_model import Lasso
rid_reg = Lasso()
modelfit(rid_reg, train, train_y)


Model Report
RMSE :  0.0181
CV Score : Mean -  0.0178 | Std -  0.0024 | Min -  0.0146 | Max - 0.0206
Amount of time spent during training the model and cross validation: 2.153 seconds


In [15]:
%%time
forest_reg = RandomForestRegressor(n_estimators=300, 
                                   random_state=1026, 
                                   min_samples_leaf=5,
                                   min_samples_split = 5,
                                   max_depth = 15,
                                   n_jobs=-1, oob_score=True)
modelfit(forest_reg, train, train_y)


Model Report
RMSE :  4.6620
CV Score : Mean -  6.1535 | Std -  1.9010 | Min -  4.3007 | Max - 10.0638
Amount of time spent during training the model and cross validation: 61.108 seconds
Wall time: 1min 1s


In [18]:
# plot_feature_importance(forest_reg, train)

In [25]:
%%time
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=421, max_depth = 15, min_samples_split = 5)
modelfit(tree_reg, train, train_y) 


Model Report
RMSE :  1.4918
CV Score : Mean -  6.0832 | Std -  1.2517 | Min -  4.7777 | Max - 8.0901
Amount of time spent during training the model and cross validation: 0.942 seconds
Wall time: 944 ms


In [26]:
from sklearn.ensemble import AdaBoostRegressor

# tree_ada = DecisionTreeRegressor(random_state = 42,max_depth = 4)

ada_reg = AdaBoostRegressor(
    tree_reg, n_estimators=150, random_state=42,learning_rate=0.019, loss='square')
modelfit(ada_reg, train, train_y)

# 181.589


Model Report
RMSE :  0.3801
CV Score : Mean -  3.9659 | Std -  0.5640 | Min -  3.2383 | Max - 4.7293
Amount of time spent during training the model and cross validation: 119.693 seconds


In [22]:
params = {'n_estimators': 100, 'max_depth': 14, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params_1 = {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth' : 14,
            'max_features': 'sqrt', 'min_samples_leaf': 15, 'min_samples_split': 5, 
            'loss': 'huber', 'random_state': 42}

# train_dummies_prepared = num_pipeline.fit_transform(train_dummies)

gdb_model = GradientBoostingRegressor(**params_1)
modelfit(gdb_model, train, train_y)


Model Report
RMSE :  6.3090
CV Score : Mean -  12.8319 | Std -  2.8306 | Min -  9.8100 | Max - 16.9810
Amount of time spent during training the model and cross validation: 385.655 seconds


In [52]:
%%time
from mlxtend.regressor import StackingRegressor

stck_reg = StackingRegressor(
    [forest_reg, tree_reg, ada_reg], 
    meta_regressor = ada_reg, 
     verbose=1)

stck_reg.fit(train, train_y)

Fitting 3 regressors...
Fitting regressor1: randomforestregressor (1/3)
Fitting regressor2: decisiontreeregressor (2/3)
Fitting regressor3: adaboostregressor (3/3)


StackingRegressor(meta_regressor=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=15,
                                                                                        min_samples_split=5,
                                                                                        random_state=421),
                                                   learning_rate=0.009,
                                                   loss='square',
                                                   n_estimators=300,
                                                   random_state=42),
                  regressors=[RandomForestRegressor(max_depth=15,
                                                    min_samples_leaf=5,
                                                    min_samples_split=5,
                                                    n_estimators=300, n_jobs=-1,
                                                    oob_score=True,
                                              

In [53]:

target = stck_reg.predict(valid)

np.sqrt(mean_squared_error(target, valid_y))

3.72873217522552

In [50]:
# plot_feature_importance(ada_reg, train)
# it seems head count and other unacceptable variables are determine it all ., 
# so its useless. 

def test_set_rmse(method):
    target = method.predict(valid)
    return print("\t the test set RMSE for \
        \n {} \n \t============================== \n".format(method),\
        np.sqrt(mean_squared_error(target, valid_y)))
test_set_rmse(tree_reg)


	 the test set RMSE for         
 DecisionTreeRegressor(max_depth=15, min_samples_split=5, random_state=421) 
 5.0024039450157884


In [51]:
test_set_rmse(ada_reg)

	 the test set RMSE for         
 AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=15,
                                                       min_samples_split=5,
                                                       random_state=421),
                  learning_rate=0.009, loss='square', n_estimators=150,
                  random_state=42) 
 3.604858128066888


In [51]:

target = forest_reg.predict(valid)

np.sqrt(mean_squared_error(target, valid_y))

6.251478370175772

# LDA \ PCA \  KPCA: 
> the feature extraction tools

In [2]:
# import pycaret

In [128]:
from mlxtend.data import iris_data
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import LinearDiscriminantAnalysis, PrincipalComponentAnalysis, RBFKernelPCA 

X, y = iris_data()

In [132]:

from mlxtend.data import iris_data
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis

X, y = iris_data()
X = standardize(X)

# PCA
pca = PrincipalComponentAnalysis(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)

# KPCA
kpca = RBFKernelPCA(gamma=15.0, n_components=3)
kpca.fit(X)
X_kpca = kpca.X_projected_

In [133]:
from sklearn.svm import SVC, LinearSVC
svc = SVC(len(np.unique(y)) + 1)
modelfit(svc, X, y)
modelfit(svc, X_pca, y)
modelfit(svc, X_kpca, y)


Model Report
RMSE :  0.1155
CV Score : Mean -  0.1667 | Std -  0.1374 | Min -  0.0000 | Max - 0.4000
Amount of time spent during training the model and cross validation: 0.047 seconds

Model Report
RMSE :  0.1633
CV Score : Mean -  0.1333 | Std -  0.1491 | Min -  0.0000 | Max - 0.4000
Amount of time spent during training the model and cross validation: 0.044 seconds

Model Report
RMSE :  0.8042
CV Score : Mean -  0.8121 | Std -  0.1432 | Min -  0.6325 | Max - 1.0000
Amount of time spent during training the model and cross validation: 0.053 seconds


In [134]:
svc2 = LinearSVC()
modelfit(svc2, X, y)
modelfit(svc2, X_pca, y)
modelfit(svc2, X_kpca, y)


Model Report
RMSE :  0.2309
CV Score : Mean -  0.1621 | Std -  0.1836 | Min -  0.0000 | Max - 0.4899
Amount of time spent during training the model and cross validation: 0.047 seconds

Model Report
RMSE :  0.2309
CV Score : Mean -  0.1955 | Std -  0.1687 | Min -  0.0000 | Max - 0.4899
Amount of time spent during training the model and cross validation: 0.046 seconds

Model Report
RMSE :  0.7211
CV Score : Mean -  0.8607 | Std -  0.1386 | Min -  0.7211 | Max - 1.0954
Amount of time spent during training the model and cross validation: 0.032 seconds


# EFS \ SFS
> feture selection

In [139]:
pd.DataFrame(X)

,0,1,2,3
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


In [160]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
 

knn = KNeighborsClassifier(n_neighbors=6)

efs1 = EFS(knn, 
           min_features=1,
           max_features=4,
           scoring='accuracy',
           print_progress=True,
           cv=5)

efs1 = efs1.fit(X, y)

print('Best accuracy score: %.2f' % efs1.best_score_)
print('Best subset (indices):', efs1.best_idx_)
print('Best subset (corresponding names):', efs1.best_feature_names_)

Features: 15/15

Best accuracy score: 0.97
Best subset (indices): (2, 3)
Best subset (corresponding names): ('2', '3')


In [159]:
svc = SVC(len(np.unique(y)) + 1)
efs2 = EFS(svc, 
           min_features=1,
           max_features=4,
           scoring='accuracy',
           print_progress=True,
           cv=5)

efs2 = efs2.fit(X, y)

print('Best accuracy score: %.2f' % efs2.best_score_)
print('Best subset (indices):', efs2.best_idx_)
print('Best subset (corresponding names):', efs2.best_feature_names_)

Features: 15/15

Best accuracy score: 0.99
Best subset (indices): (0, 1, 2, 3)
Best subset (corresponding names): ('0', '1', '2', '3')


In [170]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(knn, 
           k_features= X.shape[1], 
           forward=True, 
           floating=False, 
           verbose=0,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)
sfs1.subsets_

{1: {'feature_idx': (3,),
  'cv_scores': array([0.96]),
  'avg_score': 0.96,
  'feature_names': ('3',)},
 2: {'feature_idx': (0, 3),
  'cv_scores': array([0.96666667]),
  'avg_score': 0.9666666666666667,
  'feature_names': ('0', '3')},
 3: {'feature_idx': (0, 2, 3),
  'cv_scores': array([0.96]),
  'avg_score': 0.96,
  'feature_names': ('0', '2', '3')},
 4: {'feature_idx': (0, 1, 2, 3),
  'cv_scores': array([0.96666667]),
  'avg_score': 0.9666666666666667,
  'feature_names': ('0', '1', '2', '3')}}

In [171]:

sfs1 = SFS(svc, 
           k_features= X.shape[1], 
           forward=True, 
           floating=False, 
           verbose=0,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)
sfs1.subsets_

{1: {'feature_idx': (3,),
  'cv_scores': array([0.96]),
  'avg_score': 0.96,
  'feature_names': ('3',)},
 2: {'feature_idx': (0, 3),
  'cv_scores': array([0.96666667]),
  'avg_score': 0.9666666666666667,
  'feature_names': ('0', '3')},
 3: {'feature_idx': (0, 1, 3),
  'cv_scores': array([0.96666667]),
  'avg_score': 0.9666666666666667,
  'feature_names': ('0', '1', '3')},
 4: {'feature_idx': (0, 1, 2, 3),
  'cv_scores': array([0.98666667]),
  'avg_score': 0.9866666666666667,
  'feature_names': ('0', '1', '2', '3')}}